#Computer Vision Final Project

Details:
*   Sign Language Dataset for ASL Numbers and Operators
*   CNN for recognizing hand sign
*   OpenCV to capture hand gestures
*   use trained model to recognize hand gestures
*   recognized signs to form mathematical expression
*   evaluate expression

In [3]:
!pip install opendatasets
!pip install pandas
!pip install opencv-python
!pip install mediapipe
!pip install PyDrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 30.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [4]:
import opendatasets as od
import pandas
import cv2
import os
import numpy as np
import mediapipe as mp
import requests
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import matplotlib.pyplot as plt

In [5]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [6]:
od.download("https://www.kaggle.com/datasets/divinelavente/american-sign-language-digit-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: divinelavente
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/divinelavente/american-sign-language-digit-dataset


100%|██████████| 128M/128M [00:05<00:00, 26.8MB/s]


In [7]:
# import shutil
# directory_to_delete = '/content/'
# shutil.rmtree(directory_to_delete)
# print("Dataset deleted successfully.")

In [8]:
print("Current working directory:", os.getcwd())

print("Contents of the current directory:", os.listdir())

Current working directory: /content
Contents of the current directory: ['.config', 'american-sign-language-digit-dataset', 'sample_data']


In [9]:
base_path = "/content/american-sign-language-digit-dataset/American Sign Language Digits Dataset"

In [10]:
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")

contents = os.listdir(current_directory)
print(f"Contents of the current directory: {contents}")

for item in contents:
    item_path = os.path.join(current_directory, item)
    if os.path.isdir(item_path):
        sub_contents = os.listdir(item_path)
        if sub_contents:
            print(f"Directory '{item}' is not empty. Contains: {sub_contents}")
        else:
            print(f"Directory '{item}' is empty.")
    else:
        print(f"'{item}' is not a directory.")

Current working directory: /content
Contents of the current directory: ['.config', 'american-sign-language-digit-dataset', 'sample_data']
Directory '.config' is not empty. Contains: ['logs', 'config_sentinel', '.last_survey_prompt.yaml', 'default_configs.db', 'configurations', '.last_opt_in_prompt.yaml', '.last_update_check.json', 'gce', 'active_config']
Directory 'american-sign-language-digit-dataset' is not empty. Contains: ['American Sign Language Digits Dataset']
Directory 'sample_data' is not empty. Contains: ['README.md', 'anscombe.json', 'california_housing_train.csv', 'california_housing_test.csv', 'mnist_test.csv', 'mnist_train_small.csv']


In [11]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.7)

def extract_keypoints(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    if result.multi_hand_landmarks:
        hand_landmarks = result.multi_hand_landmarks[0]
        keypoints = []
        for lm in hand_landmarks.landmark:
            keypoints.extend([lm.x, lm.y, lm.z])
        return keypoints
    return None

imagepaths = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(('.png', '.jpg', '.jpeg')):
            imagepaths.append(os.path.join(root, file))

X = []  # keypoints
y = []  # labels

for path in imagepaths:
    img = cv2.imread(path)
    if img is None:
        print(f"Error: The image at path '{path}' could not be read.")
        continue
    keypoints = extract_keypoints(img)
    if keypoints:
        X.append(keypoints)
        try:
            folder_name = path.split("/")[-2]
            label = ''.join(filter(str.isdigit, folder_name))
            y.append(int(label))
        except ValueError as ve:
            print(f"Error: Unable to convert label to integer in path '{path}': {ve}")
            continue

X = np.array(X)
y = np.array(y)

print("Keypoints extracted: ", len(X))
print("Labels extracted: ", len(y))

# save keypoints and labels for training
np.save('X_keypoints.npy', X)
np.save('y_labels.npy', y)

Keypoints extracted:  4391
Labels extracted:  4391


In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [13]:
X_keypoints = np.load('X_keypoints.npy')
y_labels = np.load('y_labels.npy')
print(f"X_keypoints shape: {X_keypoints.shape}")
print(f"y_labels shape: {y_labels.shape}")

y_labels = y_labels.astype(int)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_keypoints, y_labels, test_size=0.2, random_state=42)

# cnn model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_keypoints.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(17, activation='softmax')  # 10 classes for digits 0-9
])

# compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# train model
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

# eval model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

X_keypoints shape: (4391, 63)
y_labels shape: (4391,)
Epoch 1/30
110/110 [==============================] - 2s 5ms/step - loss: 2.5821 - accuracy: 0.1384 - val_loss: 2.2194 - val_accuracy: 0.3868
Epoch 2/30
110/110 [==============================] - 0s 3ms/step - loss: 2.0792 - accuracy: 0.3138 - val_loss: 1.6268 - val_accuracy: 0.7213
Epoch 3/30
110/110 [==============================] - 0s 3ms/step - loss: 1.5568 - accuracy: 0.5074 - val_loss: 1.0582 - val_accuracy: 0.8237
Epoch 4/30
110/110 [==============================] - 0s 3ms/step - loss: 1.1507 - accuracy: 0.6577 - val_loss: 0.7007 - val_accuracy: 0.9101
Epoch 5/30
110/110 [==============================] - 0s 3ms/step - loss: 0.8738 - accuracy: 0.7395 - val_loss: 0.4850 - val_accuracy: 0.9306
Epoch 6/30
110/110 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.7930 - val_loss: 0.3366 - val_accuracy: 0.9386
Epoch 7/30
110/110 [==============================] - 0s 3ms/step - loss: 0.5525 - accuracy: 0

In [14]:
from tensorflow.keras.models import save_model
from google.colab import files

save_model(model, 'hand_gesture_model.h5')
files.download('hand_gesture_model.h5')

# save h5 for app

<ipython-input-14-451c0da75a47>:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, 'hand_gesture_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
# base_path = "/content/american-sign-language-digit-dataset/American Sign Language Digits Dataset/*"

In [16]:
# mp_hands = mp.solutions.hands
# hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)
# # Function to visualize keypoints on the image
# def visualize_keypoints(image, hand_landmarks):
#     if hand_landmarks:
#         for hand_landmark in hand_landmarks:
#             for lm in hand_landmark.landmark:
#                 x, y = int(lm.x * image.shape[1]), int(lm.y * image.shape[0])
#                 cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
#     return image

# # Function to extract keypoints from an image
# def extract_keypoints(image):
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     result = hands.process(image_rgb)
#     if result.multi_hand_landmarks:
#         return result.multi_hand_landmarks
#     return None

# # Get list of image paths
# imagepaths = []
# for root, dirs, files in os.walk(base_path):
#     for file in files:
#         if file.endswith(('.png', '.jpg', '.jpeg')):
#             imagepaths.append(os.path.join(root, file))

# # Loop through each image, extract keypoints, and display the image with keypoints drawn
# for path in imagepaths:
#     img = cv2.imread(path)
#     if img is None:
#         print(f"Error: The image at path '{path}' could not be read.")
#         continue
#     hand_landmarks = extract_keypoints(img)
#     if hand_landmarks:
#         img_with_keypoints = visualize_keypoints(img.copy(), hand_landmarks)
#         plt.imshow(cv2.cvtColor(img_with_keypoints, cv2.COLOR_BGR2RGB))
#         plt.axis('off')
#         plt.show()